</center><img src="coverage_workflow_image.png"  width="1000" height="300"/></center>

In [1]:
# import required python libraries
#import geopandas as gpd  # for geo-data structure
import numpy as np  # for some operation better done with numpy
import matplotlib.pyplot as plt  # for creating plots
#import shapely.geometry  # for geometry data crunching
#from shapely.geometry import Point  # specifically for point geometry data manipulation
#import keplergl
#from keplergl import KeplerGl
import datacompy
import pandas as pd
crs = {'init': 'epsg:4326'}
#from shapely.geometry import Point
import mpl_toolkits


In [11]:
# bring in the 3 to 4 different coverage dataset from FB, GSMA, ITU and Ericsson as well as the master school_geolocation

In [2]:
d_f = pd.read_csv('./data/Namibia_fb.csv',low_memory=False,encoding='utf-8')
d_i = pd.read_csv('./data/Namibia_ITU.csv',low_memory=False,encoding='utf-8')
d_e = pd.read_csv('./data/Namibia_ec.csv',low_memory=False,encoding='utf-8')
#d_g = pd.read_csv('./data/GSMA_RW_Schools_mobile_coverage_v1.csv',low_memory=False,encoding='utf-8')
s_master = pd.read_csv('./data/Namibia_school_geolocation_master_v1.csv',low_memory=False,encoding='utf-8')

In [3]:
d_e.head(2)

,LATITUDE,LONGITUDE,giga_school_id,SCHOOL_CODE,SCHOOL_NAME,education_level,CATEGORY_C,ID1,school_type,internet_speed_Mbps,...,2G_Best_Coverage_provider,3G_Best_Coverage_provider,4G_Best_Coverage_provider,5G_Best_Coverage_provider,2G_coverage_present,3G_coverage_present,4G_coverage_present,5G_coverage_present,Distance_to_nearest_tower,Data_Source
0,-17.965591,18.866897,14c9bd7b-28d3-361e-9025-5beb95ce18da,7694,Mpengu Junior Primary School,junior primary,2,Grade 0 - 7,Public,NaN,...,None,None,None,None,No,No,No,No,-1,Ericsson
1,-17.454400,15.908580,1fd5addc-46ee-377f-89fe-ee50854f7446,8227,Omutaku Primary School,primary,2,Grade 0 - 7,Public,NaN,...,MTC,MTC,TN Mobile,None,Yes,Yes,Yes,No,-1,Ericsson


In [4]:
d_e.rename(columns={'SCHOOL_CODE':'source_school_id', 'SCHOOL_NAME':'school_name', '2G_coverage':'2G_quality','3G_coverage':'3G_quality','4G_coverage':'4G_quality', 'Coverage':'coverage','Coverage_type': 'coverage_type'}, inplace=True)
#d_i.rename(columns={'name':'school_name'}, inplace=True)
#d_f.rename(columns={'school_id': 'giga_school_id'}, inplace=True)

In [56]:
# cross-profiles the sources with master data for completeness and correctness

In [6]:
# create a separate dfs
master = s_master.copy()
fb = d_f.copy()
ec = d_e.copy()
itu = d_i.copy()
#gsma = d_g.copy()
# for facebook
cross_profile_fb_master = datacompy.Compare(
master,
fb,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Facebook'
)
#for ericsson
cross_profile_e_master = datacompy.Compare(
master,
ec,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Ericsson'
)
# for ITU
cross_profile_itu_master = datacompy.Compare(
master,
itu,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='ITU'
)

#print(cross_profile_fb_master.report())
print(cross_profile_fb_master.report())
fm = open("./data/report/cross_profile_fb_master_NA_report.txt", "a")
print(cross_profile_fb_master.report(), file=fm)
fm.close()

#print(cross_profile_Ericsson_master.report())
print(cross_profile_e_master.report())
em = open("./data/report/cross_profile_ec_master_NA_report.txt", "a")
print(cross_profile_e_master.report(), file=em)
em.close()

#print(cross_profile_itu_master.report())
print(cross_profile_itu_master.report())
im = open("./data/report/cross_profile_itu_master_NA_report.txt", "a")
print(cross_profile_itu_master.report(), file=im)
im.close()

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0    Master       66  1779
1  Facebook       18  1779

Column Summary
--------------

Number of columns in common: 12
Number of columns in Master but not in Facebook: 54
Number of columns in Facebook but not in Master: 6

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 1,779
Number of rows in Master but not in Facebook: 0
Number of rows in Facebook but not in Master: 0

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 1,779

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 12
Total number of values which compare unequal: 0


DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0

In [58]:
# some colunm names standardization

In [59]:
#d_e.rename(columns={'SCHOOL_CODE':'school_id', 'SCHOOL_NAME':'school_name'}, inplace=True)
#d_i.rename(columns={'giga_schoo': 'giga_school_id','source_sch':'school_id', 'name':'school_name'}, inplace=True)
#d_g.rename(columns={'giga_schoo': 'giga_school_id'}, inplace=True)

In [60]:
# cross-profiles the sources with master data for completeness and correctness

In [61]:
# they usually don't come with "coverage" colunms, we create it at source level using the following codes

In [7]:
#d_e['coverage'] = np.where((d_e['2G_coverage']== 'None') & (d_e['3G_coverage']== 'None')& (d_e['4G_coverage']== 'None'), 'NO', 'YES')
d_f['coverage'] = np.where((d_f['percent_2G'].isnull()) & (d_f['percent_3G'].isnull())& (d_f['percent_4G'].isnull()), 'NO', 'YES')
d_f['coverage'] = np.where((d_f['percent_2G']<=0) & (d_f['percent_3G']<=0) & (d_f['percent_4G']<=0), 'NO', 'YES')
d_i['coverage'] = np.where((d_i['2G'].isnull()) & (d_i['3G'].isnull())& (d_i['4G'].isnull()), 'NO', 'YES')
d_i['coverage'] = np.where((d_i['2G']<=0) & (d_i['3G']<=0) & (d_i['4G']<=0), 'NO', 'YES')


In [63]:
# create the 2G, 3G and 4G coverage columns with binary values at source level using the following codes

In [8]:
d_f['2G_coverage'] = np.where((d_f['percent_2G'] > 0), 'YES', 'NO')
d_f['3G_coverage'] = np.where((d_f['percent_3G'] > 0), 'YES', 'NO')
d_f['4G_coverage'] = np.where((d_f['percent_4G'] > 0), 'YES', 'NO')

In [9]:
d_i['2G_coverage'] = np.where((d_i['2G'] >=1), 'YES', 'NO')
d_i['3G_coverage'] = np.where((d_i['3G'] == 1), 'YES', 'NO')
d_i['4G_coverage'] = np.where((d_i['4G'] ==1), 'YES', 'NO')

In [66]:
#d_e.rename(columns={'2G_coverage':'2G_coverage_quality', '3G_coverage':'3G_coverage_quality', '4G_coverage': '4G_coverage_quality'}, inplace=True)

In [67]:
# they usually don't come with "coverage_type" colunms, we create it at source level using the following function and codes

In [10]:
def coverage_type(df):
    if df['4G_coverage'] == 'YES':
        return "4G"
    elif df['3G_coverage'] == 'YES':
        return "3G"
    elif df['2G_coverage'] == 'YES':
        return "2G"
    else:
        return "None"

In [11]:
d_f['coverage_type'] = d_f.apply((lambda x: coverage_type(x)),axis=1)
#d_e['coverage_type'] = d_e.apply((lambda x: coverage_type(x)),axis=1)
d_i['coverage_type'] = d_i.apply((lambda x: coverage_type(x)),axis=1)


In [12]:
d_e.head(1)

,LATITUDE,LONGITUDE,giga_school_id,source_school_id,school_name,education_level,CATEGORY_C,ID1,school_type,internet_speed_Mbps,...,2G_Best_Coverage_provider,3G_Best_Coverage_provider,4G_Best_Coverage_provider,5G_Best_Coverage_provider,2G_coverage_present,3G_coverage_present,4G_coverage_present,5G_coverage_present,Distance_to_nearest_tower,Data_Source
0,-17.965591,18.866897,14c9bd7b-28d3-361e-9025-5beb95ce18da,7694,Mpengu Junior Primary School,junior primary,2,Grade 0 - 7,Public,NaN,...,None,None,None,None,No,No,No,No,-1,Ericsson


In [13]:
d_e.rename(columns={'2G_coverage_present':'2G_coverage', '3G_coverage_present':'3G_coverage','4G_coverage_present':'4G_coverage', 'Coverage':'coverage', 'Coverage_type':'coverage_type'}, inplace=True)

In [72]:
# isolating the needed colunms from the rest

In [14]:
features = ['giga_school_id','coverage','coverage_type','2G_coverage','3G_coverage','4G_coverage'] 

In [15]:
d_fc = d_f[features]
#d_ec = d_e[features]
d_it = d_i[features]
d_ec = d_e[features]

In [31]:
# more sanity checks for nan on giga_school_id

In [16]:
d_fc = d_fc[d_fc['giga_school_id'].notna()]
#d_ec = d_ec[d_ec['giga_school_id'].notna()]
d_it = d_it[d_it['giga_school_id'].notna()]
d_ec = d_ec[d_ec['giga_school_id'].notna()]

In [17]:
d_ec.loc[d_ec['2G_coverage'] == 'Yes', '2G_coverage'] = 'YES'
d_ec.loc[d_ec['3G_coverage'] == 'Yes', '3G_coverage'] = 'YES'
d_ec.loc[d_ec['4G_coverage'] == 'Yes', '4G_coverage'] = 'YES'
d_ec.loc[d_ec['coverage'] == 'Yes', 'coverage'] = 'YES'

d_ec.loc[d_ec['2G_coverage'] == 'No', '2G_coverage'] = 'NO'
d_ec.loc[d_ec['3G_coverage'] == 'No', '3G_coverage'] = 'NO'
d_ec.loc[d_ec['4G_coverage'] == 'No', '4G_coverage'] = 'NO'
d_ec.loc[d_ec['coverage'] == 'No', 'coverage'] = 'NO'


In [18]:
d_it.head()

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,d64e8b47-18bf-38e4-a27d-60a6762428b5,YES,4G,YES,YES,YES
1,dd5ff341-5190-329d-a69a-9b9f6b106009,YES,3G,YES,YES,NO
2,d482dd65-85c9-30ee-ad3c-74b7fec1eaf8,YES,3G,NO,YES,NO
3,5b2ca145-aa23-3f4f-a069-4e4e69ba01a1,YES,3G,YES,YES,NO
4,25a080d3-71cd-3940-a32f-372a7c1fc99a,YES,3G,YES,YES,NO


In [76]:
# visually comparing the sources for matches and mismatches across the columns

In [19]:
df_all = pd.concat([d_fc.set_index('giga_school_id'), d_ec.set_index('giga_school_id'),d_it.set_index('giga_school_id')], 
                   axis='columns', keys=['Facebook', 'Ericsson', 'ITU'])
df_all.head()

Facebook                            \
                                     coverage coverage_type 2G_coverage   
f5d69ac4-2be8-3e8a-bcf7-31b1b20f2cdc      YES            4G         YES   
14d4b06d-e71b-377c-b24c-945395c35068      YES            4G         YES   
be14228b-c410-30ae-9bd1-5d81d6fb3ba9      YES            4G         YES   
52a270f6-2cfc-329d-b121-23d1f43c3997      YES            4G         YES   
906d400f-b94f-3b64-9575-f37aeea15516      YES            4G         YES   

                                                             Ericsson  \
                                     3G_coverage 4G_coverage coverage   
f5d69ac4-2be8-3e8a-bcf7-31b1b20f2cdc         YES         YES      YES   
14d4b06d-e71b-377c-b24c-945395c35068         YES         YES      YES   
be14228b-c410-30ae-9bd1-5d81d6fb3ba9         YES         YES      YES   
52a270f6-2cfc-329d-b121-23d1f43c3997         YES         YES      YES   
906d400f-b94f-3b64-9575-f37aeea15516         YES         YES      YES   

                                                                            \
                                     coverage_type 2G_coverage 3G_coverage   
f5d69ac4-2be8-3e8a-bcf7-31b1b20f2cdc            4G          NO         YES   
14d4b06d-e71b-377c-b24c-945395c35068            4G         YES         YES   
be14228b-c410-30ae-9bd1-5d81d6fb3ba9            3G         YES         YES   
52a270f6-2cfc-329d-b121-23d1f43c3997            3G          NO         YES   
906d400f-b94f-3b64-9575-f37aeea15516            4G         YES         YES   

                                                      ITU                \
                                     4G_coverage coverage coverage_type   
f5d69ac4-2be8-3e8a-bcf7-31b1b20f2cdc         YES      YES            4G   
14d4b06d-e71b-377c-b24c-945395c35068         YES      YES            2G   
be14228b-c410-30ae-9bd1-5d81d6fb3ba9         YES      YES            3G   
52a270f6-2cfc-329d-b121-23d1f43c3997          NO      YES            4G   
906d400f-b94f-3b64-9575-f37aeea15516         YES      YES            4G   

                                                                          
                                     2G_coverage 3G_coverage 4G_coverage  
f5d69ac4-2be8-3e8a-bcf7-31b1b20f2cdc         YES         YES         YES  
14d4b06d-e71b-377c-b24c-945395c35068         YES          NO          NO  
be14228b-c410-30ae-9bd1-5d81d6fb3ba9         YES         YES          NO  
52a270f6-2cfc-329d-b121-23d1f43c3997         YES         YES         YES  
906d400f-b94f-3b64-9575-f37aeea15516         YES         YES         YES

In [20]:
df_final = df_all.swaplevel(axis='columns')[d_fc.columns[1:]]
df_final.tail()

coverage               coverage_type  \
                                     Facebook Ericsson  ITU      Facebook   
7f4c534c-f506-31a7-9a2a-1f335a97fe3c      YES      YES  YES            4G   
a718bb57-2231-3eba-9fe2-db3374d6e0fd      YES      YES  YES            4G   
77ac9e65-4bb9-3d4c-b120-770e600ae2dd      YES       NO  YES            4G   
c37784b5-d414-3695-8a52-9e534ca0c52a      YES      YES  YES            4G   
7f8d1e7d-f681-3f2f-9d7c-38d81b2cd413      YES       NO  YES            4G   

                                                  2G_coverage                \
                                     Ericsson ITU    Facebook Ericsson  ITU   
7f4c534c-f506-31a7-9a2a-1f335a97fe3c       3G  4G         YES      YES  YES   
a718bb57-2231-3eba-9fe2-db3374d6e0fd       4G  3G         YES      YES  YES   
77ac9e65-4bb9-3d4c-b120-770e600ae2dd     None  4G         YES       NO  YES   
c37784b5-d414-3695-8a52-9e534ca0c52a       4G  4G         YES      YES  YES   
7f8d1e7d-f681-3f2f-9d7c-38d81b2cd413     None  4G         YES      YES  YES   

                                     3G_coverage               4G_coverage  \
                                        Facebook Ericsson  ITU    Facebook   
7f4c534c-f506-31a7-9a2a-1f335a97fe3c         YES      YES  YES         YES   
a718bb57-2231-3eba-9fe2-db3374d6e0fd         YES      YES  YES         YES   
77ac9e65-4bb9-3d4c-b120-770e600ae2dd         YES      YES  YES         YES   
c37784b5-d414-3695-8a52-9e534ca0c52a         YES      YES  YES         YES   
7f8d1e7d-f681-3f2f-9d7c-38d81b2cd413         YES       NO  YES         YES   

                                                    
                                     Ericsson  ITU  
7f4c534c-f506-31a7-9a2a-1f335a97fe3c      YES  YES  
a718bb57-2231-3eba-9fe2-db3374d6e0fd      YES   NO  
77ac9e65-4bb9-3d4c-b120-770e600ae2dd      YES  YES  
c37784b5-d414-3695-8a52-9e534ca0c52a      YES  YES  
7f8d1e7d-f681-3f2f-9d7c-38d81b2cd413       NO  YES

In [22]:
def highlight_diff(data, color='lightcoral'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('Facebook', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

#df_final.style.apply(highlight_diff, axis=None)

In [81]:
# coverage comparison report

In [23]:
# create a separate dfs
fbc = d_fc.copy()
#ecc = d_ec.copy()
ituc = d_it.copy()
ecc = d_ec.copy()

# Facebook&ITU
cross_profile_fb_itu = datacompy.Compare(
fbc,
ituc,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='ITU'
)

cross_profile_ec_itu = datacompy.Compare(
ecc,
ituc,
join_columns='giga_school_id',  
df1_name='Ericsson', 
df2_name='ITU'
)


#print(cross_profile_fb_itu.report())
print(cross_profile_fb_itu.report())
fir = open("./data/report/cross_profile_fb_itu_NA_report.txt", "a")
print(cross_profile_fb_itu.report(), file=fir)
fir.close()

#print(cross_profile_Ericsson_itu.report())
print(cross_profile_ec_itu.report())
eir = open("./data/report/cross_profile_ec_itu_NA_report.txt", "a")
print(cross_profile_ec_itu.report(), file=eir)
eir.close()

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0  Facebook        6  1779
1       ITU        6  1779

Column Summary
--------------

Number of columns in common: 6
Number of columns in Facebook but not in ITU: 0
Number of columns in ITU but not in Facebook: 0

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 1,779
Number of rows in Facebook but not in ITU: 0
Number of rows in ITU but not in Facebook: 0

Number of rows with some compared columns unequal: 660
Number of rows with all compared columns equal: 1,119

Column Comparison
-----------------

Number of columns compared with some values unequal: 4
Number of columns compared with all values equal: 2
Total number of values which compare unequal: 1,740

Columns with Unequal Values or Types
------------------------------------

          Column Facebook dtype ITU dtype  

In [39]:
# merging and selecting the most frequent values across the sources on each row and colunm
# creating a master coverage dataset and merging with the master school location dataset

In [24]:
import statistics as stats
df_c = pd.concat([d_fc,d_ec,d_it])
coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x:x.value_counts().index[0])
coverage['coverage_type'] = coverage.apply((lambda x: coverage_type(x)),axis=1)
#coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x: stats.mode(x)[0][0])


In [25]:
coverage['coverage'] = np.where((coverage['2G_coverage']=='YES') | (coverage['3G_coverage']=='YES') | (coverage['4G_coverage']=='YES'), 'YES', 'NO')

In [26]:
coverage.head()

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,000c28a4-b05d-3b18-9e48-d6160e8b08b0,YES,4G,YES,YES,YES
1,0049cc97-1438-3180-bf37-f02483dc8f05,YES,4G,YES,YES,YES
2,00742d10-2587-3f97-85fc-c8f27f261718,YES,4G,YES,YES,YES
3,007c0c5f-a502-3b6c-8315-9220a8455aef,YES,4G,YES,YES,YES
4,008a562e-a511-3434-be77-ba1d9de5a106,YES,2G,YES,NO,NO


In [27]:
coverage[coverage['coverage']=='NO'].head(10)

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
7,00eb3b18-5e07-3b27-91cf-5078d9bf654e,NO,None,NO,NO,NO
25,030c8325-b5e6-306c-a718-19a209631419,NO,None,NO,NO,NO
91,0ac62d78-92bf-3709-b6bd-a02574ecdfe8,NO,None,NO,NO,NO
98,0bb0230f-1ec6-3aa6-a32c-6975db773074,NO,None,NO,NO,NO
141,114fa481-0e11-3d3c-a3e4-1dc8717828d9,NO,None,NO,NO,NO
204,1a3c8d75-d1fe-32e7-bf2a-666f00a8b9d8,NO,None,NO,NO,NO
219,1cc052a6-3d6e-3ee0-8c08-615cf9d4ae05,NO,None,NO,NO,NO
257,21815081-006d-3932-94bc-b14c30d21b4d,NO,None,NO,NO,NO
274,246fc37e-48b5-3e6d-958e-9a40a5d80f7d,NO,None,NO,NO,NO
277,24e510e6-9002-3d98-b03b-2f8431ccd8dc,NO,None,NO,NO,NO


In [97]:
#coverage.loc[coverage['coverage_type'] == 'None', 'coverage_type'] = d_gs.coverage_type

In [28]:
master.head()

,giga_school_id,source_school_id,school_name,education_level,category_c,id1,school_type,longitude,latitude,internet_speed_mbps,...,missing_education_level_flag,missing_internet_type_flag,internet_speed_outlier_flag,missing_internet_speed_mbps_flag,missing_school_type_flag,school_density_outlier_flag,dq_score,giga_school_id_seq,master_timestamp,school_archive
0,d64e8b47-18bf-38e4-a27d-60a6762428b5,8808,Ariamsvlei Primary School,primary,2,Grade 0 - 7,Public,19.8408,-28.1322,NaN,...,0,1,0,1,0,0,0.93,G_NAM_000001,45:38.5,No
1,dd5ff341-5190-329d-a69a-9b9f6b106009,7023,Bethanie Primary School,primary,2,Grade 0 - 7,Public,17.1558,-26.4986,NaN,...,0,1,0,1,0,0,0.93,G_NAM_000002,45:38.5,No
2,d482dd65-85c9-30ee-ad3c-74b7fec1eaf8,7854,Blauwes Primary School,primary,2,Grade 0 - 7,Public,18.2750,-26.1411,NaN,...,0,1,0,1,0,0,0.89,G_NAM_000003,45:38.5,No
3,5b2ca145-aa23-3f4f-a069-4e4e69ba01a1,7035,Chris Lotter Primary School,primary,2,Grade 0 - 7,Public,19.1214,-25.9336,NaN,...,0,1,0,1,0,0,0.93,G_NAM_000004,45:38.5,No
4,25a080d3-71cd-3940-a32f-372a7c1fc99a,7856,D.C. Frederick Primary School,primary,2,Grade 0 - 7,Public,17.1472,-26.5119,NaN,...,0,1,0,1,0,0,0.93,G_NAM_000005,45:38.5,No


In [29]:
master_w_coverage = master.merge(coverage, how="left", on=["giga_school_id"])
master_w_coverage.to_csv('./data/NA_master_w_coverage.csv', encoding='utf-8')
master_w_coverage_connectivity = master_w_coverage.copy()
#master_w_coverage_connectivity['connectivity_availability'] = np.where((master_w_coverage_connectivity['connectivity_type'] != 'None'), 'YES', 'NO')
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage'] == 'NO', 'connectivity'] = 'NO'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage'] == 'NO', 'coverage_type'] = 'None'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage_type'] == 'None', 'coverage'] = 'NO'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage_type'] == 'None', 'coverage_type'] = 'no coverage'
master_w_coverage_connectivity.to_csv('./data/NA_master_w_coverage_connectivity.csv', encoding='utf-8')

In [36]:
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['internet_speed_mbps'].notna(), 'connectivity'] = 'YES'

In [39]:
master_w_coverage_connectivity[master_w_coverage_connectivity['internet_speed_mbps'].notna()].head(10)

,giga_school_id,source_school_id,school_name,education_level,category_c,id1,school_type,longitude,latitude,internet_speed_mbps,...,dq_score,giga_school_id_seq,master_timestamp,school_archive,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage,connectivity
10,1c327fea-1df0-3707-b3a2-571b2ad35a7f,7871,Ernst Jager Junior combined School,combined,3,Grade 0 - 12,Public,18.740370,-28.012190,6.0,...,1.00,G_NAM_000011,45:38.5,No,YES,4G,YES,YES,YES,YES
53,01528fd0-c701-3131-bba8-cbbadfc9acff,7087,Christelike Privaatskool Hentiesbaai,primary,2,Grade 0 - 7,Private,14.287920,-22.122270,4.0,...,1.00,G_NAM_000054,45:38.5,No,YES,4G,YES,YES,YES,YES
56,7f1a8d9d-581b-3000-88a2-ef0b48dabb01,7031,Deutsche Privatschule Omaruru,primary,2,Grade 0 - 7,Private,15.944167,-21.421667,6.0,...,1.00,G_NAM_000057,45:38.5,No,YES,4G,YES,YES,YES,YES
57,ebbd2e23-2936-3d2e-a2d6-7a61ab06e842,7221,Dibasen Junior Secondary School,senior secondary,4,Grade 4 - 12,Public,15.374440,-21.365278,2.0,...,1.00,G_NAM_000058,45:38.5,No,YES,3G,YES,YES,NO,YES
61,ed9b20d4-977e-31be-8e63-8d73f4036fa4,7321,Elifas Goseb Primary School,primary,2,Grade 0 - 7,Public,15.594444,-21.982778,6.0,...,1.00,G_NAM_000062,45:38.5,No,YES,4G,YES,YES,YES,YES
62,4aea1a29-4969-3ed6-b1c0-df1fb807d19e,7825,Erongosig Primary School,primary,2,Grade 0 - 7,Public,15.595278,-21.973889,6.0,...,1.00,G_NAM_000063,45:38.5,No,YES,4G,YES,YES,YES,YES
63,8509de31-f20d-34bf-a24a-ac10419062db,7325,Festus Gonteb Junior Primary School,junior primary,2,Grade 0 - 7,Public,14.545940,-22.661120,4.0,...,1.00,G_NAM_000064,45:38.5,No,YES,4G,YES,YES,YES,YES
64,ac27102f-fc2c-3af5-bf25-6817051fd4db,7231,Flamingo Primary School,primary,2,Grade 0 - 7,Public,14.498610,-22.966670,4.0,...,1.00,G_NAM_000065,45:38.5,No,YES,4G,YES,YES,YES,YES
66,e41e7d08-4bc4-385f-b813-d180db9a89a3,7086,Headstart Montessori Elementary School,primary,2,Grade 0 - 7,Public,14.497880,-22.958510,6.0,...,1.00,G_NAM_000067,45:38.5,No,YES,4G,YES,YES,YES,YES
72,b118bc8d-7739-39fa-91cc-5204a57b8233,7226,Karibib Private Center,primary,2,Grade 0 - 7,Private,15.852890,-21.940500,4.0,...,0.96,G_NAM_000073,45:38.5,No,YES,4G,YES,YES,YES,YES


In [37]:
master_w_coverage.head(2)

,giga_school_id,source_school_id,school_name,education_level,category_c,id1,school_type,longitude,latitude,internet_speed_mbps,...,school_density_outlier_flag,dq_score,giga_school_id_seq,master_timestamp,school_archive,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,d64e8b47-18bf-38e4-a27d-60a6762428b5,8808,Ariamsvlei Primary School,primary,2,Grade 0 - 7,Public,19.8408,-28.1322,NaN,...,0,0.93,G_NAM_000001,45:38.5,No,YES,4G,YES,YES,YES
1,dd5ff341-5190-329d-a69a-9b9f6b106009,7023,Bethanie Primary School,primary,2,Grade 0 - 7,Public,17.1558,-26.4986,NaN,...,0,0.93,G_NAM_000002,45:38.5,No,YES,3G,YES,YES,NO


In [31]:
master_w_coverage_connectivity[master_w_coverage_connectivity['coverage_type']=='no coverage'].head(10)

,giga_school_id,source_school_id,school_name,education_level,category_c,id1,school_type,longitude,latitude,internet_speed_mbps,...,dq_score,giga_school_id_seq,master_timestamp,school_archive,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage,connectivity
36,905354b5-25fb-3da9-b30d-e60b5ee093cf,7867,S.C.Vries Primary School,primary,2,Grade 0 - 7,Public,17.818333,-26.459722,NaN,...,1.00,G_NAM_000037,45:38.5,No,NO,no coverage,NO,NO,NO,NO
165,eba98049-6ba5-3004-89b0-041f93f8fb31,8526,Witkop Primary School,primary,2,Grade 0 - 7,Public,17.059722,-23.785000,NaN,...,0.93,G_NAM_000166,45:38.5,No,NO,no coverage,NO,NO,NO,NO
166,d7fc941e-3186-3bb9-a9ef-6637a493d909,8531,Witkrans Primary School,primary,2,Grade 0 - 7,Public,16.415556,-23.754722,NaN,...,1.00,G_NAM_000167,45:38.5,No,NO,no coverage,NO,NO,NO,NO
213,596fb853-1864-34e7-b711-73f99360e4a8,7794,Karukuvisa Junior Primary School,junior primary,2,Grade 0 - 7,Public,19.726811,-18.952553,NaN,...,1.00,G_NAM_000214,45:38.5,No,NO,no coverage,NO,NO,NO,NO
261,3ded94f2-afcb-3642-abdd-59e97cc92a26,7649,Ncushe Junior Primary School,junior primary,2,Grade 0 - 7,Public,19.747220,-18.687780,NaN,...,1.00,G_NAM_000262,45:38.5,No,NO,no coverage,NO,NO,NO,NO
286,030c8325-b5e6-306c-a718-19a209631419,7624,Shakambu Junior Primary School,junior primary,2,Grade 0 - 7,Public,20.076110,-18.266390,NaN,...,1.00,G_NAM_000287,45:38.5,No,NO,no coverage,NO,NO,NO,NO
312,73c414a6-1db0-38c9-8749-8f2e75184c40,7640,Taratara Primary School,primary,2,Grade 0 - 7,Public,20.322222,-18.215833,NaN,...,1.00,G_NAM_000313,45:38.5,No,NO,no coverage,NO,NO,NO,NO
322,375d016a-b557-303b-a166-b49d4c870f32,7919,Zadan Junior Primary School,junior primary,2,Grade 0 - 7,Public,19.734722,-18.991667,NaN,...,1.00,G_NAM_000323,45:38.5,No,NO,no coverage,NO,NO,NO,NO
338,612774cf-8437-3f9c-9784-d56e40c02fc3,7765,Etopenkuru Junior Primary School,junior primary,2,Grade 0 - 7,Public,19.519720,-18.623610,NaN,...,1.00,G_NAM_000339,45:38.5,No,NO,no coverage,NO,NO,NO,NO
343,78040853-5f49-35c9-b0bb-e6a61284ceb1,7669,Gcaruhua Junior Primary School,junior primary,2,Grade 0 - 7,Public,18.918330,-18.008890,NaN,...,1.00,G_NAM_000344,45:38.5,No,NO,no coverage,NO,NO,NO,NO


In [33]:
master_w_coverage_connectivity.head()

,giga_school_id,source_school_id,school_name,education_level,category_c,id1,school_type,longitude,latitude,internet_speed_mbps,...,dq_score,giga_school_id_seq,master_timestamp,school_archive,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage,connectivity
0,d64e8b47-18bf-38e4-a27d-60a6762428b5,8808,Ariamsvlei Primary School,primary,2,Grade 0 - 7,Public,19.8408,-28.1322,NaN,...,0.93,G_NAM_000001,45:38.5,No,YES,4G,YES,YES,YES,NaN
1,dd5ff341-5190-329d-a69a-9b9f6b106009,7023,Bethanie Primary School,primary,2,Grade 0 - 7,Public,17.1558,-26.4986,NaN,...,0.93,G_NAM_000002,45:38.5,No,YES,3G,YES,YES,NO,NaN
2,d482dd65-85c9-30ee-ad3c-74b7fec1eaf8,7854,Blauwes Primary School,primary,2,Grade 0 - 7,Public,18.2750,-26.1411,NaN,...,0.89,G_NAM_000003,45:38.5,No,YES,4G,YES,YES,YES,NaN
3,5b2ca145-aa23-3f4f-a069-4e4e69ba01a1,7035,Chris Lotter Primary School,primary,2,Grade 0 - 7,Public,19.1214,-25.9336,NaN,...,0.93,G_NAM_000004,45:38.5,No,YES,3G,YES,YES,NO,NaN
4,25a080d3-71cd-3940-a32f-372a7c1fc99a,7856,D.C. Frederick Primary School,primary,2,Grade 0 - 7,Public,17.1472,-26.5119,NaN,...,0.93,G_NAM_000005,45:38.5,No,YES,4G,YES,YES,YES,NaN


In [34]:
master_w_coverage_connectivity['coverage'].value_counts()

YES    1724
NO       55
Name: coverage, dtype: int64

In [40]:
master_w_coverage_connectivity.to_csv('./data/NA_master_w_coverage_connectivity.csv', encoding='utf-8')